### Augmenting the Training Set
We add a *non-lateral* class to the training set in order to allow the model to automatically skip scans that are not laterals and therefore don't need to be rotated.

In [14]:
# Usual imports

from fastai.vision.all import *
import numpy as np
from PIL import Image
from pathlib import Path
import matplotlib.patches as patches
import pickle
import re
from skimage import exposure, io, color
from multiprocessing import Pool

In [3]:
folder = Path('Scans')
laterals = {d.name for d in folder.glob('*/*') if d.is_dir() and "lat" in d.name.lower()}
laterals

{'Lateral', 'Lateral Ceph', 'Lateral Skull', 'LateralCeph', 'Laterals'}

In [15]:
files = folder.rglob("*.png")
scans = []
for f in files:
    if not any([s in str(f) for s in laterals]):
        scans.append(f)

print(f"{len(scans)} non lateral scans in Scans folder")
non_lat = random.sample(scans, 512)
random.sample(non_lat, 3)

7040 non lateral scans in Scans folder


[Path('Scans/B0065/Knee/B0065HM13y02m.png'),
 Path('Scans/B3120/Knee/B3120KF07y00m.png'),
 Path('Scans/B2464/Knee/B2464KM08y00m.png')]

In [18]:
train_folder = "Train/none2"
rotations = [90, 180, 270]

def process_image(scan):
    img = Image.open(scan)
    img.save(f"{train_folder}/{scan.stem}_none.png")
    img_flip = img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
    img_flip.save(f"{train_folder}/{scan.stem}F_none.png")
    for rot in rotations:
        # img_rot = img.rotate(rot)
        img_rot = img.transpose(Image.Transpose[f"ROTATE_{rot}"])
        img_rot.save(f"{train_folder}/{scan.stem}{rot}_none.png")
        img_rot_flip = img_rot.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
        img_rot_flip.save(f"{train_folder}/{scan.stem}{rot}F_none.png")


with Pool() as p:
    p.map(process_image, non_lat)
    